# AD Mixer Drive 파이프라인 (임시 480p 변환)
이 노트북은 백엔드 큐에서 작업을 가져와 Google Drive에 있는 영상을 480p로 변환한 뒤 다시 업로드하는 임시 파이프라인입니다. 추후 AD 자동 생성 모듈을 붙일 때 기본 구조로 재사용할 수 있습니다.


## 준비 사항
1. 같은 Google Cloud 프로젝트의 **서비스 계정 JSON 키**를 `/content/service-account.json` 경로로 업로드합니다.
2. 백엔드 서버(`server`)가 로컬 혹은 클라우드에서 실행 중이어야 합니다.
3. 업로드/결과 Google Drive 폴더 ID를 백엔드 `.env`와 동일하게 노트북에도 지정합니다.
4. 변환 결과는 480p H.264 + AAC로 저장하며, Colab 런타임 환경에 따라 ffmpeg 설치 시간이 추가될 수 있습니다.


In [1]:
%pip install --quiet moviepy requests google-api-python-client google-auth


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: C:\Users\HS\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [ ]:
import json
import os
import pathlib
import requests
from datetime import datetime

from google.oauth2 import service_account
from googleapiclient.discovery import build
from googleapiclient.http import MediaFileUpload, MediaIoBaseDownload

API_BASE_URL = os.environ.get('API_BASE_URL', 'http://localhost:4000/api')
SERVICE_ACCOUNT_FILE = pathlib.Path('/content/service-account.json')
UPLOAD_FOLDER_ID = os.environ.get('GOOGLE_DRIVE_UPLOAD_FOLDER_ID')
RESULT_FOLDER_ID = os.environ.get('GOOGLE_DRIVE_RESULT_FOLDER_ID', UPLOAD_FOLDER_ID)

assert SERVICE_ACCOUNT_FILE.exists(), 'service-account.json 파일을 먼저 업로드하세요.'
assert API_BASE_URL, 'API_BASE_URL 환경 변수를 설정하세요.'
assert UPLOAD_FOLDER_ID, 'GOOGLE_DRIVE_UPLOAD_FOLDER_ID 값이 필요합니다.'

SCOPES = ['https://www.googleapis.com/auth/drive']

def build_drive_client():
    creds = service_account.Credentials.from_service_account_file(
        SERVICE_ACCOUNT_FILE, scopes=SCOPES
    )
    return build('drive', 'v3', credentials=creds)

drive = build_drive_client()
print('Drive client ready, API base =', API_BASE_URL)


In [ ]:
import tempfile
from pathlib import Path
from moviepy.editor import VideoFileClip


def fetch_next_job():
    res = requests.get(f'{API_BASE_URL}/jobs/next', timeout=30)
    res.raise_for_status()
    return res.json()


def update_status(job_id, status, note=''):
    payload = {'status': status, 'note': note}
    res = requests.post(f'{API_BASE_URL}/jobs/{job_id}/progress', json=payload, timeout=30)
    res.raise_for_status()
    return res.json()


def mark_failure(job_id, note):
    res = requests.post(f'{API_BASE_URL}/jobs/{job_id}/fail', json={'note': note}, timeout=30)
    res.raise_for_status()
    return res.json()


def download_drive_file(file_id, destination):
    request = drive.files().get_media(fileId=file_id)
    with open(destination, 'wb') as target:
        downloader = MediaIoBaseDownload(target, request)
        done = False
        while not done:
            status, done = downloader.next_chunk()
            if status:
                print(f'Downloading... {int(status.progress() * 100)}%')


def upload_to_drive(file_path, name, folder_id):
    file_metadata = {'name': name, 'parents': [folder_id] if folder_id else None}
    media = MediaFileUpload(file_path, mimetype='video/mp4')
    created = drive.files().create(body=file_metadata, media_body=media, fields='id').execute()
    drive.permissions().create(fileId=created['id'], body={'role': 'reader', 'type': 'anyone'}).execute()
    return drive.files().get(fileId=created['id'], fields='id, webViewLink, webContentLink').execute()


def process_to_480p(input_path, output_path):
    clip = VideoFileClip(input_path)
    clip_resized = clip.resize(height=480)
    clip_resized.write_videofile(
        output_path,
        codec='libx264',
        audio_codec='aac',
        remove_temp=True,
        bitrate='2000k'
    )
    clip.close()


def handle_job(job):
    print('처리 시작:', job['id'], job['sourceDriveId'])
    update_status(job['id'], 'processing', '원본 다운로드 중')
    with tempfile.TemporaryDirectory() as tmpdir:
        source_path = os.path.join(tmpdir, job['sourceFileName'])
        result_path = os.path.join(tmpdir, f"{Path(job['sourceFileName']).stem}_480p.mp4")
        download_drive_file(job['sourceDriveId'], source_path)
        update_status(job['id'], 'generating', '480p 변환 중')
        process_to_480p(source_path, result_path)
        update_status(job['id'], 'rendering', 'Drive 업로드 중')
        info = upload_to_drive(result_path, os.path.basename(result_path), RESULT_FOLDER_ID)
        requests.post(
            f"{API_BASE_URL}/jobs/{job['id']}/result",
            json={
                'resultDriveId': info['id'],
                'resultWebViewLink': info['webViewLink'],
                'resultDownloadLink': info['webContentLink'],
                'note': '480p 변환 완료'
            },
            timeout=30
        ).raise_for_status()
        print('완료:', job['id'], info['webViewLink'])


In [ ]:
try:
    job = fetch_next_job()
    if not job:
        print('큐에 대기 중인 작업이 없습니다.')
    else:
        handle_job(job)
except Exception as exc:
    print('처리 중 오류:', exc)
    if 'job' in locals() and job:
        mark_failure(job['id'], str(exc))
